# Book Reading Propensity as a Recommender System: 
# Collaborative Filtering (CF) & Matrix Factorization (MF)

In [23]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import time

from statistics import mean

from surprise import Reader, Dataset, SVD, SVDpp, BaselineOnly, NMF, accuracy
from surprise.dataset import DatasetAutoFolds
from surprise.model_selection import train_test_split, GridSearchCV, PredefinedKFold

from sklearn.model_selection import train_test_split as skl_train_test_split

from propensity_utils import *

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
random_state = 42

## Read the Train/Validation Data

## Matrix Factorization with the Scikit Suprise: Model Selection
References:
 * https://surpriselib.com/
 * https://surprise.readthedocs.io/en/stable/index.html

### Building the Train Dataset

In [11]:
train_val_path = './data/goodreads_2016_train_val.csv'
train = DatasetBuilder(train_val_path)

### Grid Search: SVD

In [33]:
# Full Search Grid:
# Note: In the spirit of rapid incremental improvements, 
# we break down the search grid into smaller subgrids.
# The next two cells illustrate one of many such experiments

svd_param_grid_full = {
    "n_factors": [32, 64, 128],
    "n_epochs": [10, 20, 50, 100],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.1, 0.5]
}

In [55]:
%%time
svd_param_grid = {
    "n_factors": [128],
    "n_epochs": [50, 100],
    "lr_all": [0.005],
    "reg_all": [0.1]
}

gs = GridSearchCV(SVD, svd_param_grid, measures=["rmse", "mae", "fcp"], cv=3, n_jobs=3, joblib_verbose=1)

gs.fit(train.dataset)
print('*'*50)
print('Best RMSE score, with hyperparameters')
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print('*'*50)
print('Best MAE score, with hyperparameters')
print(gs.best_score["mae"])
print(gs.best_params["mae"])
print('*'*50)
print('Best FCP score, with hyperparameters')
print(gs.best_score["fcp"])
print(gs.best_params["fcp"])
print('*'*50)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


**************************************************
Best RMSE score, with hyperparameters
1.1549409149407357
{'n_factors': 128, 'n_epochs': 100, 'lr_all': 0.005, 'reg_all': 0.1}
**************************************************
Best MAE score, with hyperparameters
0.8318418249949565
{'n_factors': 128, 'n_epochs': 100, 'lr_all': 0.005, 'reg_all': 0.1}
**************************************************
Best FCP score, with hyperparameters
0.6187826379436273
{'n_factors': 128, 'n_epochs': 100, 'lr_all': 0.005, 'reg_all': 0.1}
**************************************************
CPU times: user 50 s, sys: 1min 20s, total: 2min 10s
Wall time: 11min 2s


[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed: 10.3min finished


In [8]:
# gs.cv_results

### Grid Search: NMF

In [33]:
# Full Search Grid:
# Note: In the spirit of rapid incremental improvements, 
# we break down the search grid into smaller subgrids.
# The next two cells illustrate one of many such experiments

nmf_param_grid_full = {
    "n_factors": [32, 64, 128],
    "n_epochs": [10, 20, 50, 100],
    "biased": [False, True]
}

In [6]:
%%time
nmf_param_grid = {
    "n_factors": [128],
    "n_epochs": [10],
    "biased": [False, True]
}

gs = GridSearchCV(NMF, nmf_param_grid, measures=["rmse", "mae", "fcp"], cv=3, n_jobs=3, joblib_verbose=1)

gs.fit(train.dataset)
print('*'*50)
print('Best RMSE score, with hyperparameters')
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print('*'*50)
print('Best MAE score, with hyperparameters')
print(gs.best_score["mae"])
print(gs.best_params["mae"])
print('*'*50)
print('Best FCP score, with hyperparameters')
print(gs.best_score["fcp"])
print(gs.best_params["fcp"])
print('*'*50)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


**************************************************
Best RMSE score, with hyperparameters
1.3348718206119399
{'n_factors': 128, 'n_epochs': 10, 'biased': False}
**************************************************
Best MAE score, with hyperparameters
0.9369168304480354
{'n_factors': 128, 'n_epochs': 10, 'biased': False}
**************************************************
Best FCP score, with hyperparameters
0.3638301603132758
{'n_factors': 128, 'n_epochs': 10, 'biased': True}
**************************************************
CPU times: user 41.7 s, sys: 36.7 s, total: 1min 18s
Wall time: 11min 19s


[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed: 10.8min finished


In [7]:
gs.cv_results

{'split0_test_rmse': array([1.33480871, 1.91178115]),
 'split1_test_rmse': array([1.33507148, 1.92561447]),
 'split2_test_rmse': array([1.33473527, 1.94094462]),
 'mean_test_rmse': array([1.33487182, 1.92611342]),
 'std_test_rmse': array([0.00014433, 0.01191116]),
 'rank_test_rmse': array([1, 2]),
 'split0_test_mae': array([0.93674739, 1.45747351]),
 'split1_test_mae': array([0.93707775, 1.47037776]),
 'split2_test_mae': array([0.93692536, 1.49048659]),
 'mean_test_mae': array([0.93691683, 1.47277929]),
 'std_test_mae': array([0.000135  , 0.01358409]),
 'rank_test_mae': array([1, 2]),
 'split0_test_fcp': array([0.33495661, 0.37628206]),
 'split1_test_fcp': array([0.33938084, 0.35712743]),
 'split2_test_fcp': array([0.32837866, 0.358081  ]),
 'mean_test_fcp': array([0.33423871, 0.36383016]),
 'std_test_fcp': array([0.00452022, 0.00881342]),
 'rank_test_fcp': array([2, 1]),
 'mean_fit_time': array([189.97589628, 252.0327901 ]),
 'std_fit_time': array([ 5.72445159, 20.29270348]),
 'mean_t

### Evaluate & Compare the Selected MF Algorithms
 * Each type of the CF algorithms is assumed to have been already optimized via the grid search cross-validation procedure.
 * Now we compare these optimized algorithms against each other.

In [5]:
%%time
train_eval_run = TrainEvalAlgos(train)

n_factors = 128
print(f'n_factors = {n_factors}')
print('*'*50)
svd = SVD(**{'n_factors': n_factors, 'n_epochs': 100, 
             'lr_all': 0.005, 'reg_all': 0.1,
             'random_state':random_state})
train_eval_run.addAlgorithm(svd)

nmf = NMF(**{'n_factors': n_factors, 'n_epochs': 100, 
             'biased': False,
             'random_state':random_state})
train_eval_run.addAlgorithm(nmf)

bsl_options = {'method': 'als',
               'n_epochs': 10,
               'reg_u': 15,
               'reg_i': 10
               }
als = BaselineOnly(bsl_options=bsl_options)
train_eval_run.addAlgorithm(als)

train_eval_run.train_and_evaluate()
print('*'*50)

n_factors = 128
**************************************************
RMSE: 0.7871
MAE:  0.6010
FCP:  0.6133
-----------
SVD
-----------
      Metrics - RMSE: 0.7871062836643817, MAE: 0.6010177803199526, FCP: 0.6133032823338325
-----------
RMSE: 0.8314
MAE:  0.6167
FCP:  0.6152
-----------
NMF
-----------
      Metrics - RMSE: 0.83139702193419, MAE: 0.6166544099703551, FCP: 0.6151914220918498
-----------
Estimating biases using als...
RMSE: 0.8157
MAE:  0.6339
FCP:  0.5873
-----------
BaselineOnly
-----------
      Metrics - RMSE: 0.8156643711387394, MAE: 0.6339487358014173, FCP: 0.58726104542583
-----------
**************************************************
CPU times: user 10min 42s, sys: 2min 40s, total: 13min 22s
Wall time: 15min 3s


In [4]:
del train_eval_run
del train

### Notes on the Model Evaluation & Comparison
 * The SVD (Singular Value Decomposition) algorithm proved to be marginally better than the NMF (Non-negative Matrix Factorization), when evaluated with several accuracy metrics: RMSE (root mean square error), MAE (mean absolute error), and FCP (Fraction of Concordant Pairs).
 * SVD also proved to be much faster to train in comparison to the NMF, while using the same number of epochs.
 * It is worth noting that the "Baseline Only" algorithm (i.e., an algorithm predicting the baseline estimate for given user and item: global mean, adjusted with user/item biases) was not too far behind. This suggests that perhaps our "perfect model" would benefit from a hybrid approach, when the ranks that capture the user-item interaction, are accompanied by additional, "external" features (e.g., complementary data about a user, or an item).

## Estimate the Confidence of the Propensity Model's Predictions

 * Our trained recommender can be easily converted into a propensity model, which is in essense, a binary classifier: it would ultimately generate a "yes or no" answer when asked if a reader X would like a book Y.
 * We define the confidence for the results of such propensity model as a probability that the model will make a correct prediction for any randomly selected user-item pair.
 * If the validation set size is $n$ and the number of correct predictions is $n_c$, then a point estimate of the the confidence $C$ is $$\hat{C} = \frac{n_c}{n}$$
 * We can estimate the value of the confidence on the train set, using the model we trained, which is a common practice for recommender systems. In order to minimize a bias of such estimate, we can use a K-fold cross-validation procedure. 
 
### K-Fold Cross-Validation Procedure

1. Get the Dataset as a Pandas DF
2. FOR-loop: K iterations where we reshuffle the data, and split it into train/validation sets (folds)
    1. Convert the train/validation sets into Surprise datasets
    2. Train the model on the train set
    3. Predict the ratings for the validation set
    4. Convert the rating column into a binary classification (1/0 = "yes/no") column
    5. Esimate the confidence as a probability of a correct prediction

In [7]:
df_train_val = pd.read_csv(train_val_path)
print(f'Data size: {df_train_val.shape[0]: >8,d}.')
df_train_val.head()

Data size: 3,491,950.


,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,10673579,5
1,7504b2aee1ecb5b2872d3da381c6c91e,29069989,2
2,7504b2aee1ecb5b2872d3da381c6c91e,8882815,3
3,7504b2aee1ecb5b2872d3da381c6c91e,6693332,4
4,7504b2aee1ecb5b2872d3da381c6c91e,4588949,4


In [25]:
%%time

# Number of cross-validation folds
K = 10
# MF model parameters
n_factors = 128
n_epochs = 10

confidence_estimates = []

reader = Reader(rating_scale=(1, 5))

for k in range(1,K+1):
    start_time = time.time()
    # Shuffle & Split the Data
    df_train_k, df_val_k = skl_train_test_split(df_train_val, test_size=0.2, random_state=random_state+k)
    # Create a Surprise Trainset
    train = Dataset.load_from_df(df_train_k[['user_id', 'book_id', 'rating']], reader).build_full_trainset()
    # Train the MF model
    algo = SVD(**{'n_factors': n_factors, 'n_epochs': n_epochs, 'lr_all': 0.005, 'reg_all': 0.1})
    algo.fit(train)
    # Predict the ratings for the validation set
    df_val_k['rating_predicted'] = df_val_k.apply(lambda x: algo.predict(x['user_id'], 
                                                                  x.book_id, 
                                                                  r_ui=x.rating, 
                                                                  verbose=False).est, axis=1)
    # Convert the rating column into a binary classification (1/0 = "yes/no") column
    df_val_k['would_recommend'] = df_val_k['rating'].apply(lambda x: 1 if x >= 4 else 0)
    df_val_k['would_recommend_pred'] = df_val_k['rating_predicted'].apply(lambda x: 1 if x >= 4 else 0)
    # Make a point esimate for the confidence as a probability of a correct prediction
    conf = df_val_k[df_val_k['would_recommend_pred'] == df_val_k['would_recommend']]['rating'].\
                count()/df_val_k.shape[0]
    confidence_estimates.append(conf)
    time_elapsed = time.time() - start_time
    print(f'Fold {k:>2d}: C = {conf:.5f}. Time elapsed: {time_elapsed/60:>5.2f} minutes')
    del df_train_k, df_val_k

confidence_on_train = mean(confidence_estimates)
print('*'*60)
print(f'Estimated Confidence (on a {k}-fold CV): {confidence_on_train:.2f}')
print('*'*60)


Fold  1: C = 0.69473. Time elapsed:  0.72 minutes
Fold  2: C = 0.69440. Time elapsed:  0.81 minutes
Fold  3: C = 0.69577. Time elapsed:  0.79 minutes
Fold  4: C = 0.69434. Time elapsed:  0.83 minutes
Fold  5: C = 0.69409. Time elapsed:  0.83 minutes
Fold  6: C = 0.69375. Time elapsed:  0.78 minutes
Fold  7: C = 0.69421. Time elapsed:  0.86 minutes
Fold  8: C = 0.69440. Time elapsed:  1.51 minutes
Fold  9: C = 0.69350. Time elapsed:  1.39 minutes
Fold 10: C = 0.69334. Time elapsed:  0.92 minutes
************************************************************
Estimated Confidence (on a 10-fold CV): 0.69
************************************************************
CPU times: user 6min 53s, sys: 1min 37s, total: 8min 30s
Wall time: 9min 27s


## Train & Evaluate a Propensity Model

In [25]:
# reader = Reader(rating_scale=(1, 5))
# ratings = pd.read_csv(train_val_path)
# train = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader).build_full_trainset()
# del ratings
# # train.df.head()

In [26]:
# n_factors = 128
# n_epochs = 100
# svd = SVD(**{'n_factors': n_factors, 'n_epochs': n_epochs, 'lr_all': 0.005, 'reg_all': 0.1})
# svd.fit(train)

## Inference

 * A quote from the assignment: "*The input for the model will be (user_id, item_id) with other feature data and the output will be (positive or negative, confidence value)*"
 * __Propencity model and "cold starts":__
     * Whenever both the __user and the item are "known"__ (i.e., previously observed), we use previously selected model that showed the best performance on the train set.
     * Whenever we encounter a __"cold start" case (a user and/or item is new, previously unobserved)__, we will use the __baseline model__, i.e. an average rating, with corrections for a user/item bias (if we happen to have that data).
 * __Estimating the confidence:__ 
     * We can use the accuracy of the binary ("positive/negative") predictions on the train set, as an estimate of the confidence
     * As an incremental improvement, we can adjust the confidence based on the number of ratings provided by the user (the more we know about the user's preferences, the more accurately we can make a recommendation) 

In [35]:
n_factors = 128
n_epochs = 100

reader = Reader(line_format="user item rating", sep=",", skip_lines=1, rating_scale=(1, 5))

# folds_files is a list of tuples containing file paths:
train_file = train_val_path
test_file = test_path
folds_files = [(train_file, test_file) ]

data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo = SVD(**{'n_factors': n_factors, 'n_epochs': n_epochs, 
                  'lr_all': 0.005, 'reg_all': 0.1, 
                  'random_state':random_state})
    
    algo.fit(trainset)
    predictions = svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)
    accuracy.fcp(predictions, verbose=True)

RMSE: 0.8599
MAE:  0.6681
FCP:  0.5855


### Read the Test Set

In [36]:
test_path = './data/goodreads_2016_test.csv'

df_test = pd.read_csv(test_path)
print(f'Data size: {df_test.shape[0]: >8,d}.')
df_test.head()

Data size:  321,965.


,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,29058155,3
1,8842281e1d1347389f2ab93d60773d4d,76620,5
2,01ec1a320ffded6b2dd47833f2c8e4fb,30853358,5
3,4b3636a043e5c99fa27ac897ccfa1151,34084,4
4,afc070543f19028dc7e7f084a0079f72,18131,5


### Example: Make a prediction for a single user-item pair

In [42]:
uid = str('8842281e1d1347389f2ab93d60773d4d')  # raw user id (as in the ratings file). They are **strings**!
iid = str(29058155)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=3, verbose=False)
pred.est

4.353580280247132

### Predict the ratings for the entire test set

In [47]:
df_test['rating_predicted'] = df_test.apply(lambda x: algo.predict(x['user_id'], 
                                                                  x.book_id, 
                                                                  r_ui=x.rating, 
                                                                  verbose=False).est, axis=1)
                                            
df_test.head()

,user_id,book_id,rating,rating_predicted
0,8842281e1d1347389f2ab93d60773d4d,29058155,3,4.304734
1,8842281e1d1347389f2ab93d60773d4d,76620,5,4.304734
2,01ec1a320ffded6b2dd47833f2c8e4fb,30853358,5,4.310409
3,4b3636a043e5c99fa27ac897ccfa1151,34084,4,3.557498
4,afc070543f19028dc7e7f084a0079f72,18131,5,4.240893


### Get the prtedicted binary recommendations (`yes/no` $\leftrightarrow$ `1/0`)

In [48]:
df_test['would_recommend'] = df_test['rating'].apply(lambda x: 1 if x >= 4 else 0)
df_test['would_recommend_pred'] = df_test['rating_predicted'].apply(lambda x: 1 if x >= 4 else 0)
                                            
df_test.head()

,user_id,book_id,rating,rating_predicted,would_recommend,would_recommend_pred
0,8842281e1d1347389f2ab93d60773d4d,29058155,3,4.304734,0,1
1,8842281e1d1347389f2ab93d60773d4d,76620,5,4.304734,1,1
2,01ec1a320ffded6b2dd47833f2c8e4fb,30853358,5,4.310409,1,1
3,4b3636a043e5c99fa27ac897ccfa1151,34084,4,3.557498,1,0
4,afc070543f19028dc7e7f084a0079f72,18131,5,4.240893,1,1


### Estimate the accuracy (percentage of the correct guesses) on the test set

In [51]:
accuracy = df_test[df_test['would_recommend_pred'] == df_test['would_recommend']]['rating'].\
                count()/df_test.shape[0]

print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 65.85%


### Estimate the bias of the trained recommender on the test set

In [54]:
avg_rating_test = df_test['rating'].mean()
avg_predicted_rating_test = df_test['rating_predicted'].mean()

print(f'Avg. Rating:           {avg_rating_test:.2f}')
print(f'Avg. Predicted Rating: {avg_predicted_rating_test:.2f}')

Avg. Rating:           4.03
Avg. Predicted Rating: 4.02


### Notes on the estimated accuracy and bias
 * When evaluated on the test set, the trained SVD model demonstrates a low bias.
 * On the other hand, the estimated accuracy of 65.85% suggests that our model suffers from a considerable variance. 
 * With `MAE = 0.6681`, one can see that a substantial number of user-item recommendations get misclassified near the decision boundary ("no" corresponds to `rank < 4`; "yes" is for `rank >= 4`).
 * This leads us to a conclusion that the model's performance could be improved with the use of one of data resampling techniques. This approach has a potential of lowering the variance, thereby boosting the overall accuracy.

### Estimating the bias near the decision boundary

In [6]:
df_test[3 <= df_test['rating'] <= 4]['rating'].mean()

NameError: name 'df_test' is not defined

In [6]:
df_test[3 <= df_test['rating'] <= 4]['rating_predicted'].mean()

NameError: name 'df_test' is not defined